# Content Translation Article Deletion Ratios, across all wikis

**Last updated on 19 February 2023**

[TASK: T351883](https://phabricator.wikimedia.org/T351883)

# Contents

1. [Overview](#Overview)
2. [Data Gathering](#Data-Gathering)
3. [Analysis](#Analysis)
    * [Current Quarter - FY23 Q2](#Current-Quarter)
    * [Previous Quarter - FY23 Q1](#Previous-Quarter)
4. [Formatting](#Formatting)

# Overview

## Purpose
The purpose of this analysis is to identify and list the number of wikis where the deletion rate of articles created with content translation is higher than the deletion rate for articles created with other tools. Specifically, we want to answer the following questions:
* How many wikis have translations deleted more often than regular articles?
* Which are these wikis?
* Has the number of those wikis reduced compared to the previous period?
* How high is the highest deletion ratio a wiki has for translations?
* This analysis will be used as a baseline to assess the evolution of deletion rates as improvements are made.

## Summary
* The deletion rate for CX created articles (2.88%) is significantly less than that of non-CX created articles (13.21%).
    * This was the lowest deletion rate for CX created articles after December 2021.
* There were [4 wikis](https://www.mediawiki.org/wiki/Content_translation/Deletion_statistics_comparison#October_through_December_2023_(Q2_2023)) had at least 2% higher deletion rate of CX created articles than the ones that were not. 
    * Two of these wikis had higher deletion rates for CX created articles during the last quarter as well.
    * Kurdish (kuwiki) has been on the list for the last four quarters, and Occitan (ocwiki) for the last three quarters.

# Data-Gathering

## Setup

In [7]:
import numpy as np
import pandas as pd
import wmfdata as wmf

pd.options.display.max_columns = None
from IPython.display import clear_output

import importlib
import warnings

import data_functions as dtf
import formatting_functions as ftf

In [8]:
importlib.reload(dtf)
importlib.reload(ftf)

<module 'formatting_functions' from '/srv/home/kcv-wikimf/gitref/content-translation-deletion-stats/formatting_functions.py'>

In [9]:
spark_session = wmf.spark.get_active_session()

if type(spark_session) == type(None):
    spark_session = wmf.spark.create_custom_session(
        master="yarn",
        app_name='cx-funnel-entrypoints',
        spark_config={
            "spark.driver.memory": "4g",
            "spark.dynamicAllocation.maxExecutors": 64,
            "spark.executor.memory": "16g",
            "spark.executor.cores": 4,
            "spark.sql.shuffle.partitions": 256,
            "spark.driver.maxResultSize": "2g"
        }
    )

spark_session.sparkContext.setLogLevel("ERROR")

clear_output()

spark_session

## run query

In [10]:
currq_dates = dtf.generate_quarters(2023)['Q2']
prevq_dates = dtf.generate_quarters(2023)['Q1']

In [11]:
%%time

warnings.filterwarnings('ignore')

deletion_stats_currq_all = dtf.query_deletion_stats(currq_dates)
deletion_stats_prevq_all = dtf.query_deletion_stats(prevq_dates)

CPU times: user 942 ms, sys: 220 ms, total: 1.16 s
Wall time: 4min 25s


# Analysis

## Current-Quarter

In [12]:
# overal deletion percent
dtf.overall_deletion_pct(deletion_stats_currq_all, period='FY23-Q2', pr=True)

During FY23-Q2, overall percentage of articles that were deleted,
	- created using the Content Translation tool: 2.88%
	- created without using the Content Translation Tool: 13.21%


In [13]:
# deletion ratio by wiki
deletion_stats_currq = dtf.generate_ratios_by_wiki(deletion_stats_currq_all)

In [33]:
print(f'During FY23-Q2, across all wikis where more than 15 articles have been created with the Content Translation tool,\n\
there were {deletion_stats_currq.query("""deletion_pct_diff <= -2""").shape[0]} wikis where the deletion rate of articles created \
using CX was atleast 2% higher than articles created without using CX.')

During FY23-Q2, across all wikis where more than 15 articles have been created with the Content Translation tool,
there were 4 wikis where the deletion rate of articles created using CX was atleast 2% higher than articles created without using CX.


In [34]:
# wikis with high deletion ratio
currq_high_deletion_ratio = deletion_stats_currq.query("""deletion_pct_diff <= -2""").sort_values('deletion_pct_diff')
currq_high_deletion_ratio

,created_cx,created_non_cx,deleted_cx,deleted_non_cx,deleted_cx_pct,deleted_non_cx_pct,deletion_pct_diff
wiki_db,,,,,,,
kuwiki,22,15023,3,24,13.64,0.16,-13.48
svwiki,104,12910,19,1618,18.27,12.53,-5.74
ocwiki,45,1053,3,22,6.67,2.09,-4.58
dewiki,535,53710,89,6865,16.64,12.78,-3.86


## Previous-Quarter

In [26]:
# overal deletion percent
dtf.overall_deletion_pct(deletion_stats_prevq_all, period='FY23-Q1', pr=True)

During FY23-Q1, overall percentage of articles that were deleted,
	- created using the Content Translation tool: 5.22%
	- created without using the Content Translation Tool: 12.71%


In [27]:
# deletion ratio by wiki
deletion_stats_prevq = dtf.generate_ratios_by_wiki(deletion_stats_prevq_all)

In [35]:
print(f'During FY22-Q4, across all wikis where more than 15 articles have been created with the Content Translation tool,\n\
there were {deletion_stats_prevq.query("""deletion_pct_diff < -2""").shape[0]} wikis where the deletion rate of articles created \
using CX was atleast 2% higher than articles created without using CX.')

During FY22-Q4, across all wikis where more than 15 articles have been created with the Content Translation tool,
there were 14 wikis where the deletion rate of articles created using CX was atleast 2% higher than articles created without using CX.


In [36]:
# wikis with high deletion ratio

prevq_high_deletion_ratio = deletion_stats_prevq.query("""deletion_pct_diff < -2""").sort_values('deletion_pct_diff')
prevq_high_deletion_ratio

,created_cx,created_non_cx,deleted_cx,deleted_non_cx,deleted_cx_pct,deleted_non_cx_pct,deletion_pct_diff
wiki_db,,,,,,,
aswiki,24,426,10,29,41.67,6.81,-34.86
kawiki,77,1326,33,234,42.86,17.65,-25.21
ltwiki,43,4126,13,272,30.23,6.59,-23.64
lawiki,16,603,4,49,25.00,8.13,-16.87
cebwiki,18,840,3,63,16.67,7.50,-9.17
ocwiki,36,794,3,14,8.33,1.76,-6.57
kuwiki,65,1023,5,18,7.69,1.76,-5.93
brwiki,21,2865,1,21,4.76,0.73,-4.03
lvwiki,35,3321,3,152,8.57,4.58,-3.99


In [37]:
# wikis that had high deletion rates for articles that have been created with CX compared articles that have not been created using CX
wikis_high_deletion_ratio = np.intersect1d(currq_high_deletion_ratio.index.values, prevq_high_deletion_ratio.index.values)
wikis_high_deletion_ratio

array(['kuwiki', 'ocwiki'], dtype=object)

## Formatting
for publication on Meta-Wiki at [Content translation/Deletion statistics comparison](https://www.mediawiki.org/wiki/Content_translation/Deletion_statistics_comparison)

In [38]:
currq_wtable = currq_high_deletion_ratio.copy()

In [39]:
# format the percentage columns

percentage_columns = ['deleted_cx_pct', 'deleted_non_cx_pct', 'deletion_pct_diff']
currq_wtable[percentage_columns] = currq_wtable[percentage_columns]/100

currq_wtable = (
    currq_wtable
    .assign(
        deleted_cx_pct = ftf.format_percent('deleted_cx_pct', currq_wtable),
        deleted_non_cx_pct = ftf.format_percent('deleted_non_cx_pct', currq_wtable),
        deletion_pct_diff = ftf.format_percent('deletion_pct_diff', currq_wtable)
    )
    .reset_index()
)

In [40]:
# rename columns
columns_rename_map = {
    'wiki_db': 'Wikipedia',
    'created_cx': 'Created CX Articles', 
    'created_non_cx': 'Created non-CX Articles', 
    'deleted_cx': 'Deleted CX Articles', 
    'deleted_non_cx': 'Deleted non-CX Articles',
    'deleted_cx_pct': 'CX Articles Deletion Ratio', 
    'deleted_non_cx_pct': 'Non-CX Articles Deletion Ratio', 
    'deletion_pct_diff': 'Deletion Ratio Difference'
}

currq_wtable.rename(columns_rename_map, axis=1, inplace=True)

In [41]:
# create a multi-level column
column_arrays = [
    np.array(['Wikipedia'] + ['Created Articles'] * 2 + ['Deleted Articles'] * 2 + ['Deletion Ratios'] * 3),
    currq_wtable.columns.to_numpy()
]

currq_wtable.columns = pd.MultiIndex.from_arrays(column_arrays)

currq_wtable.head()

Wikipedia    Created Articles                            Deleted Articles  \
  Wikipedia Created CX Articles Created non-CX Articles Deleted CX Articles   
0    kuwiki                  22                   15023                   3   
1    svwiki                 104                   12910                  19   
2    ocwiki                  45                    1053                   3   
3    dewiki                 535                   53710                  89   

                                     Deletion Ratios  \
  Deleted non-CX Articles CX Articles Deletion Ratio   
0                      24                     13.64%   
1                    1618                     18.27%   
2                      22                      6.67%   
3                    6865                     16.64%   

                                                            
  Non-CX Articles Deletion Ratio Deletion Ratio Difference  
0                          0.16%                   -13.48%  
1                         12.53%                    -5.74%  
2                          2.09%                    -4.58%  
3                         12.78%                    -3.86%

In [42]:
# add footnote (as superscript) for wikis that had high deletion ratio for article created using CX during the last quarter as well
currq_wtable[('Wikipedia', 'Wikipedia')] = currq_wtable[('Wikipedia', 'Wikipedia')].apply(lambda x:ftf.add_footnote(x, wikis_high_deletion_ratio))

In [44]:
table_headers = [
    'Wikipedias with higher deletion ratios for articles created with Content Translation',
    'Reviewed Time Period: October through December 2023 (FY 23 Q2)'
]

table_footers = [
    '<sup>1</sup> Excludes Wikipedias with 15 or fewer articles created with Content Translation during the reviewed time period.',
    '<sup>2</sup> Also identified in the prior quarter as a wiki with a higher deletion ratio for articles created with Content Translation.'
]

In [45]:
# to be published at https://www.mediawiki.org/wiki/Content_translation/Deletion_statistics_comparison
print(ftf.dataframe_to_mediawiki(currq_wtable, table_headers, table_footers))

{| class='wikitable'
! colspan='8' | Wikipedias with higher deletion ratios for articles created with Content Translation
! colspan='8' | Reviewed Time Period: October through December 2023 (FY 23 Q2)
|-
colspan='1' | Wikipedia !! colspan='2' | Created Articles !! colspan='2' | Deleted Articles !! colspan='3' | Deletion Ratios
colspan='1' | Wikipedia !! colspan='1' | Created CX Articles !! colspan='1' | Created non-CX Articles !! colspan='1' | Deleted CX Articles !! colspan='1' | Deleted non-CX Articles !! colspan='1' | CX Articles Deletion Ratio !! colspan='1' | Non-CX Articles Deletion Ratio !! colspan='1' | Deletion Ratio Difference
|-
| kuwiki<sup>2</sup> || 22 || 15023 || 3 || 24 || 13.64% || 0.16% || -13.48%
|-
| svwiki || 104 || 12910 || 19 || 1618 || 18.27% || 12.53% || -5.74%
|-
| ocwiki<sup>2</sup> || 45 || 1053 || 3 || 22 || 6.67% || 2.09% || -4.58%
|-
| dewiki || 535 || 53710 || 89 || 6865 || 16.64% || 12.78% || -3.86%
|-
| style="text-align:left;" colspan='8' | <sup>1</sup